## 基于yolov5的数独检测
### 0.环境安装

In [3]:
!git clone https://github.com/pk5ls20/sudoku.git
!mv sudoku/* .
%pip install -qr requirements.txt
!git clone https://github.com/ultralytics/yolov5.git

Cloning into 'sudoku'...
remote: Enumerating objects: 28, done.
remote: Counting objects: 100% (28/28), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 28 (delta 7), reused 21 (delta 4), pack-reused 0
Unpacking objects: 100% (28/28), 25.15 MiB | 11.69 MiB/s, done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.9 MB/s eta 0:00:00
Cloning into 'yolov5'...
remote: Enumerating objects: 15598, done.
remote: Counting objects: 100% (205/205), done.
remote: Compressing objects: 100% (151/151), done.
remote: Total 15598 (delta 98), reused 115 (delta 54), pack-reused 15393
Receiving objects: 100% (15598/15598), 14.64 MiB | 28.55 MiB/s, done.
Resolving deltas: 100% (10626/10626), done.


### 1.提取数独

In [30]:
import os
import torch
import datetime
import cv2
model = torch.hub.load('ultralytics/yolov5', 'custom', path='detect_sudoku.pt')
input_path = 'sudoku_pic'
output_path = 'sudoku_pic/extract'
timex = lambda :datetime.datetime.now()
for file_name in os.listdir(input_path):
    if file_name.endswith('.jpg') or file_name.endswith('.png'):
        print(f"正在处理{file_name}", end='')
        img = cv2.imread(os.path.join(input_path, file_name))
        # 使用YOLOv5检测
        results = model(img)
        # 得到置信度最高的数独检测结果
        sudoku_detection = None
        for result in results.pred[0]:
            if result[-1] == 0 and result[-2] > 0.8:
                sudoku_detection = result
                break
        # 提取数独
        if sudoku_detection is not None:
            xmin, ymin, xmax, ymax, confidence = sudoku_detection[:5]
            sudoku = img[int(ymin):int(ymax), int(xmin):int(xmax)]
            cv2.imwrite(os.path.join(output_path, f"extract_{file_name}"), sudoku)
            print(f"...提取成功！")
        else:
            print(f"...{file_name}未检测到数独！")
print(f"Done on {timex()}")

Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2023-4-27 Python-3.9.16 torch-2.0.0+cu118 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


requirements: /root/.cache/torch/hub/requirements.txt not found, check failed.
正在处理1.png...提取成功！
Done on 2023-04-27 03:29:15.493029


### 2.提取数独中的数字

In [31]:
# Iterate over images in input folder
fp = list(os.listdir(output_path))
for file_name in fp:
    if file_name.split('_')[0] != 'extract':
        continue
    # Load image
    img = cv2.imread(os.path.join(output_path, file_name))
    # Get dimensions of image
    height, width, _ = img.shape
    # Calculate size of each small image
    size = int(height / 9)
    # Iterate over rows and columns of small images
    for row in range(9):
        for col in range(9):
            # Calculate coordinates of small image
            x1 = col * size
            y1 = row * size
            x2 = x1 + size
            y2 = y1 + size
            # Crop small image from main image
            small_img = img[y1:y2, x1:x2]
            # Save small image to output folder
            small_img_file_name = '{}_{}.png'.format(os.path.splitext(file_name)[0], row * 9 + col)
            cv2.imwrite(os.path.join(output_path, small_img_file_name), small_img)
print(f"Done on {timex()}")

Done on 2023-04-27 03:29:24.696708


### 3.单个数字图片预处理
在识别单个数字之前，需要对图片进行预处理。
受限于训练模型，进行二值化+去黑线的预处理可以大幅度提高识别准确率

In [32]:
from PIL import Image, ImageOps
import os

# 读取图片并转换为黑白图
folder_path = 'sudoku_pic/extract'
for filename in os.listdir(folder_path):
    if len(filename.split('_')) != 3:
        continue
    img = Image.open(f"{folder_path}/{filename}").convert('L')
    # 获取图片的宽度和高度
    width, height = img.size
    cl = []
    # 遍历每一行，如果整行像素点>=80%部分不是白色，则将该行像素点全部转换为白色
    for y in range(height):
        pixels = [img.getpixel((x, y)) for x in range(width)]
        white_pixels = sum(1 for pixel in pixels if pixel == 255)
        if white_pixels < width*0.2:
            for x in range(width):
                cl.append((x, y))
    # 遍历每一列，如果整列像素点>=80%部分不是白色，则将该列像素点全部转换为白色
    for x in range(width):
        pixels = [img.getpixel((x, y)) for y in range(height)]
        white_pixels = sum(1 for pixel in pixels if pixel == 255)
        if white_pixels < height*0.2:
            for y in range(height):
                cl.append((x, y))
    # 将所有的白色像素点转换为黑色
    for x, y in cl:
        img.putpixel((x, y), 255)
    # 反转图片颜色
    img = ImageOps.invert(img)
    img.save(f"{folder_path}/ok/ok_{filename}")
print(f"Done on {timex()}")

Done on 2023-04-27 03:29:28.393855


### 4.识别数字并转化为数独


In [56]:
from torchvision.transforms import ToTensor, Resize
from pathlib import Path
import sys
from models.experimental import attempt_load
from utils.general import non_max_suppression
from utils.torch_utils import select_device
import numpy as np
from PIL import Image
global sudoku_

sys.path.insert(0, str(Path('yolov5')))

def load_model(model_path):
    device = select_device()
    model = attempt_load(model_path, device)
    model.eval()
    return model

def predict(model, image_path):
    device = select_device()
    img = Image.open(image_path).convert("RGB")
    img = Resize((128, 128))(img)
    img_tensor = ToTensor()(img).unsqueeze(0).to(device)
    pred = model(img_tensor)[0]
    # Apply non-max suppression
    results = non_max_suppression(pred, conf_thres=0.25, iou_thres=0.45, classes=None, agnostic=False)
    return results, img.size

def process_images(model_path, image_folder):
    global sudoku_
    model = load_model(model_path)
    sudoku = np.zeros((9, 9), dtype=np.int32)
    for img_name in sorted(os.listdir(image_folder)):
        if img_name.startswith("ok_extract_"):
            row = int(img_name.split("_")[3].split('.')[0]) // 9
            col = int(img_name.split("_")[3].split('.')[0]) % 9
            img_path = os.path.join(image_folder, img_name)
            results, img_size = predict(model, img_path)
            if len(results) > 0 and len(results[0]) > 0:
                most_likely_class = int(results[0][0][5].item())
                sudoku[row, col] = most_likely_class
        else:
            continue
    sudoku_=sudoku
process_images("detect_number.pt", "sudoku_pic/extract/ok")
print(sudoku_)
print(f"Done on {timex()}")

YOLOv5 🚀 2023-4-27 Python-3.9.16 torch-2.0.0+cu118 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 157 layers, 7037095 parameters, 0 gradients
YOLOv5 🚀 2023-4-27 Python-3.9.16 torch-2.0.0+cu118 CUDA:0 (Tesla T4, 15102MiB)

YOLOv5 🚀 2023-4-27 Python-3.9.16 torch-2.0.0+cu118 CUDA:0 (Tesla T4, 15102MiB)

YOLOv5 🚀 2023-4-27 Python-3.9.16 torch-2.0.0+cu118 CUDA:0 (Tesla T4, 15102MiB)

YOLOv5 🚀 2023-4-27 Python-3.9.16 torch-2.0.0+cu118 CUDA:0 (Tesla T4, 15102MiB)

YOLOv5 🚀 2023-4-27 Python-3.9.16 torch-2.0.0+cu118 CUDA:0 (Tesla T4, 15102MiB)

YOLOv5 🚀 2023-4-27 Python-3.9.16 torch-2.0.0+cu118 CUDA:0 (Tesla T4, 15102MiB)

YOLOv5 🚀 2023-4-27 Python-3.9.16 torch-2.0.0+cu118 CUDA:0 (Tesla T4, 15102MiB)

YOLOv5 🚀 2023-4-27 Python-3.9.16 torch-2.0.0+cu118 CUDA:0 (Tesla T4, 15102MiB)

YOLOv5 🚀 2023-4-27 Python-3.9.16 torch-2.0.0+cu118 CUDA:0 (Tesla T4, 15102MiB)

YOLOv5 🚀 2023-4-27 Python-3.9.16 torch-2.0.0+cu118 CUDA:0 (Tesla T4, 15102MiB)

YOLOv5 🚀 2023-4-27 Python-3.9.16 torch-2.0.

[[0 0 3 0 0 2 0 0 6]
 [0 0 0 0 1 8 0 3 0]
 [7 1 0 0 0 0 0 0 2]
 [0 0 0 0 0 0 7 0 0]
 [3 5 0 0 6 0 0 0 0]
 [0 0 6 4 0 3 0 1 0]
 [0 0 0 0 0 0 9 4 0]
 [0 0 5 9 0 0 0 0 0]
 [9 0 2 6 0 0 0 0 0]]
Done on 2023-04-27 03:45:20.362784


### 5.求解数独


In [60]:
import numpy as np
global sudoku_2
def solve_sudoku_from_np_array(input_array):
    global sudoku_2
    input_array = input_array.copy() 
    def dfs(x, y):
        if x == 9:
            return True
        if y == 9:
            return dfs(x + 1, 0)
        if input_array[x][y]:
            return dfs(x, y + 1)
        
        id = xy_id[x][y]
        for k in range(1, 10):
            if hang[x][k] or lie[y][k] or kuai[id][k]:
                continue
            input_array[x][y] = k
            hang[x][k] = 1
            lie[y][k] = 1
            kuai[id][k] = 1
            
            if dfs(x, y + 1):
                return True
            
            input_array[x][y] = 0
            hang[x][k] = 0
            lie[y][k] = 0
            kuai[id][k] = 0
        
        return False

    hang = np.zeros((9, 10), dtype=int)
    lie = np.zeros((9, 10), dtype=int)
    kuai = np.zeros((9, 10), dtype=int)

    xy_id = np.array([
        [0, 0, 0, 1, 1, 1, 2, 2, 2],
        [0, 0, 0, 1, 1, 1, 2, 2, 2],
        [0, 0, 0, 1, 1, 1, 2, 2, 2],
        [3, 3, 3, 4, 4, 4, 5, 5, 5],
        [3, 3, 3, 4, 4, 4, 5, 5, 5],
        [3, 3, 3, 4, 4, 4, 5, 5, 5],
        [6, 6, 6, 7, 7, 7, 8, 8, 8],
        [6, 6, 6, 7, 7, 7, 8, 8, 8],
        [6, 6, 6, 7, 7, 7, 8, 8, 8]
    ])

    for i in range(9):
        for j in range(9):
            if input_array[i][j] != 0:
                num = input_array[i][j]
                hang[i][num] = 1
                lie[j][num] = 1
                id = xy_id[i][j]
                kuai[id][num] = 1

    dfs(0, 0)

    sudoku_2=input_array
print(sudoku_)
solve_sudoku_from_np_array(sudoku_)
print(sudoku_2)

[[0 0 3 0 0 2 0 0 6]
 [0 0 0 0 1 8 0 3 0]
 [7 1 0 0 0 0 0 0 2]
 [0 0 0 0 0 0 7 0 0]
 [3 5 0 0 6 0 0 0 0]
 [0 0 6 4 0 3 0 1 0]
 [0 0 0 0 0 0 9 4 0]
 [0 0 5 9 0 0 0 0 0]
 [9 0 2 6 0 0 0 0 0]]
[[5 8 3 7 4 2 1 9 6]
 [2 6 9 5 1 8 4 3 7]
 [7 1 4 3 9 6 8 5 2]
 [4 9 1 8 2 5 7 6 3]
 [3 5 7 1 6 9 2 8 4]
 [8 2 6 4 7 3 5 1 9]
 [6 3 8 2 5 7 9 4 1]
 [1 7 5 9 3 4 6 2 8]
 [9 4 2 6 8 1 3 7 5]]


### 6.填回原图

In [61]:
import cv2
import numpy as np

def fill_sudoku_numbers(original_image_path, old_array, new_array):
    old_array = np.array(old_array)
    new_array = np.array(new_array)
    print(old_array)
    print(new_array)
    # 加载原始图像
    original_image = cv2.imread(original_image_path, cv2.IMREAD_GRAYSCALE)

    # 计算每个单元格的大小
    cell_width = original_image.shape[1] // 9
    cell_height = original_image.shape[0] // 9

    # 设置字体
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = (cell_width + cell_height) / 100
    font_thickness = int((cell_width + cell_height) / 25)

    for i in range(9):
        for j in range(9):
            if old_array[i][j] == 0 and new_array[i][j] != 0:
                # 计算数字在图像中的坐标
                x = cell_width * j + cell_width // 2
                y = cell_height * i + cell_height // 2
                print(x,y)
                # 在原始图像上填充数字
                cv2.putText(original_image, str(new_array[i][j]), (x, y), font, font_scale, (0, 0, 255), font_thickness, cv2.LINE_AA)

    # 保存填充后的图像
    filled_image_path = "/content/sudoku_pic/extract/filled_extract_1.png"
    cv2.imwrite(filled_image_path, original_image)

    return filled_image_path

original_image_path = "/content/sudoku_pic/extract/extract_1.png"
fill_sudoku_numbers(original_image_path,sudoku_,sudoku_2)


[[0 0 3 0 0 2 0 0 6]
 [0 0 0 0 1 8 0 3 0]
 [7 1 0 0 0 0 0 0 2]
 [0 0 0 0 0 0 7 0 0]
 [3 5 0 0 6 0 0 0 0]
 [0 0 6 4 0 3 0 1 0]
 [0 0 0 0 0 0 9 4 0]
 [0 0 5 9 0 0 0 0 0]
 [9 0 2 6 0 0 0 0 0]]
[[5 8 3 7 4 2 1 9 6]
 [2 6 9 5 1 8 4 3 7]
 [7 1 4 3 9 6 8 5 2]
 [4 9 1 8 2 5 7 6 3]
 [3 5 7 1 6 9 2 8 4]
 [8 2 6 4 7 3 5 1 9]
 [6 3 8 2 5 7 9 4 1]
 [1 7 5 9 3 4 6 2 8]
 [9 4 2 6 8 1 3 7 5]]
45 45
135 45
315 45
405 45
585 45
675 45
45 136
135 136
225 136
315 136
585 136
765 136
225 227
315 227
405 227
495 227
585 227
675 227
45 318
135 318
225 318
315 318
405 318
495 318
675 318
765 318
225 409
315 409
495 409
585 409
675 409
765 409
45 500
135 500
405 500
585 500
765 500
45 591
135 591
225 591
315 591
405 591
495 591
765 591
45 682
135 682
405 682
495 682
585 682
675 682
765 682
135 773
405 773
495 773
585 773
675 773
765 773


'/content/sudoku_pic/extract/filled_extract_1.png'